In [1]:
import cv2
import tensorflow as tf
import argparse
import numpy as np
import os
import pdb
import time
import matplotlib.pyplot as plt
import sys
print(tf.__version__)
import scipy.io as sio
import math

############### Global Parameters ###############
# path
video_list  = sio.loadmat('trainlist1.mat')
test_list = sio.loadmat('testlist1.mat')

train_path = './experiments/hmdb-51/rgb_data_split1/train/'

train_path_2 = './experiments/hmdb-51/flow_data_split1/train/'

test_path = './experiments/hmdb-51/rgb_data_split1/test/'

test_path_2 = './experiments/hmdb-51/flow_data_split1/test/'

# Model Path
default_model_path = './multi_model/multi_att_flow_1_ff/final_model'
save_path = './multi_model/multi_att_flow_1_ff/'

# Output Path
output_path = './multi_att_flow_1_res_2/'

# Train Test Split
train_num = video_list['video'][0].shape[0]-1
test_num = test_list['video'][0].shape[0]-1

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
tf.test.is_gpu_available()

/usr/local/lib/python2.7/dist-packages/requests/__init__.py:83: RequestsDependencyWarning: Old version of cryptography ([1, 2, 3]) may cause slowdown.
  warnings.warn(warning, RequestsDependencyWarning)


1.10.0


True

In [2]:
video_list ['video'][0][1][0]

array([u'April_09_brush_hair_u_nm_np1_ba_goo_1'], dtype='<U37')

In [3]:
############## Train Parameters #################
# Parameters
learning_rate = 0.00005
n_epochs = 100
batch_size = 1
# Network Parameters
n_input = 1024 # input dimension
n_hidden = 1024 # hidden layer num of multi_head
n_classes = 51 # has 51 classes
n_frames = 32

In [4]:
def squared_dist(A, B):
    assert A.shape.as_list() == B.shape.as_list()

    row_norms_A = tf.reduce_sum(tf.square(A), axis=1)
    row_norms_A = tf.reshape(row_norms_A, [-1, 1])  # Column vector.

    row_norms_B = tf.reduce_sum(tf.square(B), axis=1)
    row_norms_B = tf.reshape(row_norms_B, [1, -1])  # Row vector.

    return row_norms_A - 2 * tf.matmul(A, tf.transpose(B)) + row_norms_B

In [5]:
def normalize(inputs, 
              epsilon = 1e-8,
              scope="ln",
              reuse=None):
    '''Applies layer normalization.
    
    Args:
      inputs: A tensor with 2 or more dimensions, where the first dimension has
        `batch_size`.
      epsilon: A floating number. A very small number for preventing ZeroDivision Error.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
      
    Returns:
      A tensor with the same shape and data dtype as `inputs`.
    '''
    with tf.variable_scope(scope, reuse=reuse):
        inputs_shape = inputs.get_shape()
        params_shape = inputs_shape[-1:]
    
        mean, variance = tf.nn.moments(inputs, [-1], keep_dims=True)
        beta= tf.Variable(tf.zeros(params_shape))
        gamma = tf.Variable(tf.ones(params_shape))
        normalized = (inputs - mean) / ( (variance + epsilon) ** (.5) )
        outputs = gamma * normalized + beta
        
    return outputs

In [6]:
def multihead_attention(queries, 
                        keys,
                        qf,
                        wkeys, 
                        num_units=None, 
                        num_heads=8, 
                        dropout_rate=0,
                        is_training=True,
                        causality=False,
                        trainable=True,
                        scope="multihead_attention", 
                        reuse=None):
    '''Applies multihead attention.
    
    Args:
      queries: A 3d tensor with shape of [N, T_q, C_q].
      keys: A 3d tensor with shape of [N, T_k, C_k].
      num_units: A scalar. Attention size.
      dropout_rate: A floating point number.
      is_training: Boolean. Controller of mechanism for dropout.
      causality: Boolean. If true, units that reference the future are masked. 
      num_heads: An int. Number of heads.
      scope: Optional scope for `variable_scope`.
      reuse: Boolean, whether to reuse the weights of a previous layer
        by the same name.
        
    Returns
      A 3d tensor with shape of (N, T_q, C)  
    '''
    with tf.variable_scope(scope, reuse=reuse):
        # Set the fall back option for num_units
        if num_units is None:
            num_units = queries.get_shape().as_list[-1]
        
        # Linear projections
        Q = tf.contrib.layers.fully_connected(queries, num_units,trainable=trainable) # (N, T_q, C)
        K = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable ) # (N, T_k, C)
        V = tf.contrib.layers.fully_connected(keys, num_units,trainable=trainable) # (N, T_k, C)
        
        QF = tf.contrib.layers.fully_connected(qf, num_units,trainable=trainable) # (N, T_q, C)
        KF = tf.contrib.layers.fully_connected(qf, num_units,trainable=trainable)
        
        Q1 = tf.reshape(Q,(1,n_frames,num_units))
        K1 = tf.reshape(K,(1,n_frames,num_units))        
        V1 = tf.reshape(V,(1,n_frames,num_units))
        
        
        # Split and concat
        Q_ = tf.concat(tf.split(Q1, num_heads, axis = 2),axis =0) # (h*N, T_q, C/h) 
        K_ = tf.concat(tf.split(K1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        V_ = tf.concat(tf.split(V1, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        
        
        QF_ = tf.concat(tf.split(QF, num_heads, axis = 2),axis =0) # (h*N, T_q, C/h) 
        KF_ = tf.concat(tf.split(KF, num_heads, axis = 2),axis =0) # (h*N, T_k, C/h) 
        
        # Multiplication
        #outputs = tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
       # output_1 = tf.matmul(Q_, wkeys) # (h*N, T_q, T_k)
        #outputs  = tf.matmul(output_1, tf.transpose(K_, [0, 2, 1])) # (h*N, T_q, T_k)
        #output_2 = tf.matmul(QF, tf.transpose(KF, [0, 2, 1]))
        output_1  = output_1 = tf.matmul(QF_, wkeys) # (h*N, T_q, T_k)
        outputs  = tf.multiply(tf.matmul(Q_, tf.transpose(K_, [0, 2, 1])),tf.matmul(output_1, tf.transpose(KF_, [0, 2, 1])))
        #outputs  = output_2 + outputs
        
        # Scale
        outputs = outputs / (K_.get_shape().as_list()[-1] ** 0.5)

                
        # Key Masking
        key_masks = tf.sign(tf.abs(tf.reduce_sum(keys, axis=-1))) # (N, T_k)
        key_masks = tf.tile(key_masks, [num_heads, 1]) # (h*N, T_k)
        key_masks = tf.tile(tf.expand_dims(key_masks, 1), [1, tf.shape(queries)[1], 1]) # (h*N, T_q, T_k)
       # 
        paddings = tf.ones_like(outputs)*(-2**32+1)
        outputs = tf.where(tf.equal(key_masks, 0), paddings, outputs) # (h*N, T_q, T_k)
          
        # Causality = Future blinding
        if causality:
            diag_vals = tf.ones_like(outputs[0, :, :]) # (T_q, T_k)
            tril = tf.contrib.linalg.LinearOperatorTriL(diag_vals).to_dense() # (T_q, T_k)
            masks = tf.tile(tf.expand_dims(tril, 0), [tf.shape(outputs)[0], 1, 1]) # (h*N, T_q, T_k)
  # 
            paddings = tf.ones_like(masks)*(-2**32+1)
            outputs = tf.where(tf.equal(masks, 0), paddings, outputs) # (h*N, T_q, T_k)
  
        # Activation
        outputs = tf.nn.softmax(outputs) # (h*N, T_q, T_k)
        matt    = outputs
        
        
        # Query Masking
        query_masks = tf.sign(tf.abs(tf.reduce_sum(queries, axis=-1))) # (N, T_q)
        query_masks = tf.tile(query_masks, [num_heads, 1]) # (h*N, T_q)
        query_masks = tf.tile(tf.expand_dims(query_masks, -1), [1, 1, tf.shape(keys)[1]]) # (h*N, T_q, T_k)
        outputs *= query_masks # broadcasting. (N, T_q, C)
          
        # Dropouts
        outputs = tf.contrib.layers.dropout(outputs, keep_prob=dropout_rate, is_training=tf.convert_to_tensor(is_training))
               
        # Weighted sum
        outputs = tf.matmul(outputs, V_) # ( h*N, T_q, C/h)
        
        # Restore shape
        outputs = tf.concat(tf.split(outputs, num_heads,axis = 0),axis =2 ) # (N, T_q, C)
              
        # Residual connection
        outputs += queries
              
        # Normalize
        outputs = normalize(outputs) # (N, T_q, C)
 
    return outputs, matt

In [7]:
def build_model():

    # tf Graph input
    x  = tf.placeholder("float", [None, n_frames , n_input])
    y  = tf.placeholder("float", [None, n_classes])
    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes], mean=0.0, stddev=0.01)),
        'att_wk': tf.Variable(tf.random_normal([8,n_input/8, n_input/8], mean=0.0, stddev=0.02),trainable=True),

    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes], mean=0.0, stddev=0.01))


    }
    # init loss 
    loss = 0.0  
    # Mask 
    regularizer = tf.contrib.layers.l1_regularizer(scale=0.0005)
    reg_term    = tf.contrib.layers.apply_regularization(regularizer,[weights['att_wk']])
    # Start creat graph
    X = tf.reshape(x,(1,n_frames,n_input))
    for i in range(0,n_frames):
        xs = X[0,i,:]
        xf = tf.cast(tf.spectral.fft(tf.cast(xs,dtype=tf.complex64)),tf.float32)
        if i == 0:
                soft_xf      = tf.reshape(xf,(1,1,n_input)) 
        else:
                temp_soft_xf = tf.reshape(xf,(1,1,n_input))
                soft_xf      = tf.concat([soft_xf,temp_soft_xf],axis = 1)

    # 1 rank approximation of attention weighting  
    wkeys1 = weights['att_wk']
        
        
    # Multi attention   
    multi_att, matt =  multihead_attention(queries=X, keys=X, wkeys = wkeys1, qf = soft_xf, num_units=n_input, 
                                                        num_heads=8, 
                                                        dropout_rate=0.9,
                                                        is_training=1,
                                                        causality=False, 
                                                        scope="self_attention_1")

    for i in range(n_frames):                  
        with tf.variable_scope('model',reuse=tf.AUTO_REUSE):
            multi_ahead = tf.reshape(multi_att[0,i,:],(1,n_hidden))
            with tf.variable_scope("fc_m") as fc_m:
                fc_m = tf.contrib.layers.fully_connected(multi_ahead,n_hidden, activation_fn=tf.nn.relu)
                fc_m_drop = tf.nn.dropout(fc_m, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]   
            
            with tf.variable_scope("fc_m2") as fc_m2:
                fc_m2 = tf.contrib.layers.fully_connected(fc_m_drop,n_hidden, activation_fn=tf.nn.relu)
                fc_m2_drop = tf.nn.dropout(fc_m2, 0.8)
                fc_m_variables =  [v for v in tf.global_variables() if v.name.startswith(fc_m.name)]

            # FC to output classification
            pred = tf.matmul(fc_m2_drop , weights['out']) + biases['out']

            # save the predict of each time step
            if i == 0:
                soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
 
            else:
                temp_soft_pred = tf.reshape((tf.nn.softmax(pred)),(1,n_classes))
                soft_pred   = tf.concat([soft_pred,temp_soft_pred],axis =0)



            # negative example
            yc = tf.reshape(y[0,:],(1,n_classes))
            neg_loss = tf.nn.softmax_cross_entropy_with_logits(labels = yc,logits = pred) # Softmax loss
            temp_loss = tf.reduce_mean(neg_loss)
            loss = tf.add(loss, temp_loss)
            loss = tf.add(loss,reg_term)

      
    # Define loss and optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss/n_frames) # Adam Optimizer

    return x,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt

In [8]:
tf.reset_default_graph()
x,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
# mkdir folder for saving model
if os.path.isdir(save_path) == False:
        os.mkdir(save_path)
# Initializing the variables
init = tf.global_variables_initializer()
# Launch the graph
sess.run(init)
saver = tf.train.Saver()
#saver = tf.train.import_meta_graph(save_path +'model-40.meta')
#saver.restore(sess,save_path+'model-40')
epoch = 1
epoch_loss = np.zeros((train_num,1),dtype = float)
n_batchs = np.arange(1,train_num+1)
np.random.shuffle(n_batchs)
tStart_epoch = time.time()
matt.get_shape

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<bound method Tensor.get_shape of <tf.Tensor 'self_attention_1/Reshape_4:0' shape=(8, 32, 32) dtype=float32>>

In [9]:
# Keep training until reach max iterations
# start training
for epoch in range(n_epochs):
#  chose batch randomly
    epoch_loss = np.zeros((train_num,1),dtype = float)
    n_batchs = np.arange(1,train_num+1)
    np.random.shuffle(n_batchs)
    tStart_epoch = time.time()
    k = 0
    for batch in n_batchs:
        
        # load features and labels
        file_name = video_list['video'][0][batch][0][0]
        batch_data = np.load(train_path_2 + file_name +'.npy').item()
        batch_xs = batch_data['feat']
        
        batch_ys = batch_data['label']
        totalframe = batch_xs.shape
        totalframe = totalframe[0]
        
        # reshape input batch 
        batch_xc = np.zeros((1,n_frames,batch_xs.shape[1]))

        batch_yc = np.zeros((1,n_frames,n_classes))                   
        batch_xc[0,:,:] = batch_xs
  
        batch_zc = np.zeros((1,n_frames,2))
        
        # feed-forward
        _,batch_loss = sess.run([optimizer,loss], feed_dict={x: batch_xc, y: batch_ys})

        epoch_loss[batch-1] = batch_loss
        # Debug per batch
        k = k + 1

    # print per epoch
    print ("Epoch:", epoch+1, " done. Loss:", np.mean(epoch_loss))
    tStop_epoch = time.time()
    print ("Epoch Time Cost:", round(tStop_epoch - tStart_epoch,2), "s")
    sys.stdout.flush()
    if (epoch+1) %10 == 0:
        saver.save(sess,save_path+"model", global_step = epoch+1)
print ("Optimization Finished!")
saver.save(sess, save_path+"final_model")

('Epoch:', 1, ' done. Loss:', 69.44375468016456)
('Epoch Time Cost:', 210.52, 's')
('Epoch:', 2, ' done. Loss:', 28.578351269859446)
('Epoch Time Cost:', 157.56, 's')
('Epoch:', 3, ' done. Loss:', 21.59964502971796)
('Epoch Time Cost:', 159.5, 's')
('Epoch:', 4, ' done. Loss:', 17.639337256938557)
('Epoch Time Cost:', 158.59, 's')
('Epoch:', 5, ' done. Loss:', 14.541935188237826)
('Epoch Time Cost:', 158.46, 's')
('Epoch:', 6, ' done. Loss:', 12.192520694020937)
('Epoch Time Cost:', 157.14, 's')
('Epoch:', 7, ' done. Loss:', 10.517862504217163)
('Epoch Time Cost:', 159.11, 's')
('Epoch:', 8, ' done. Loss:', 9.148409292121952)
('Epoch Time Cost:', 158.45, 's')
('Epoch:', 9, ' done. Loss:', 7.5608074178843)
('Epoch Time Cost:', 158.14, 's')
('Epoch:', 10, ' done. Loss:', 6.576546755774611)
('Epoch Time Cost:', 157.21, 's')
('Epoch:', 11, ' done. Loss:', 5.644508292749535)
('Epoch Time Cost:', 158.66, 's')
('Epoch:', 12, ' done. Loss:', 4.616192279143729)
('Epoch Time Cost:', 157.23, 's')

('Epoch:', 98, ' done. Loss:', 0.5394576958678903)
('Epoch Time Cost:', 192.71, 's')
('Epoch:', 99, ' done. Loss:', 1.4065320285595067)
('Epoch Time Cost:', 192.48, 's')
('Epoch:', 100, ' done. Loss:', 0.596598991903438)
('Epoch Time Cost:', 192.84, 's')
Optimization Finished!


'./multi_model/multi_att_flow_1_ff/final_model'

In [10]:
tf.reset_default_graph()
att_res = {}
x,y,optimizer,loss,soft_pred,fc_m_variables,multi_att,matt= build_model()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.InteractiveSession(config=tf.ConfigProto(allow_soft_placement=True,gpu_options=gpu_options))
init = tf.global_variables_initializer()
sess.run(init)

saver = tf.train.Saver()

# restore model
print(default_model_path)
saver.restore(sess, default_model_path)
top1_list = []
for num_batch in range(0,test_num+1): # 0, test_num = 0 : test_num-1
    file_name = test_list['video'][0][num_batch][0][0]
    test_all_data = np.load(test_path_2 + file_name+'.npy').item()
  
    print(file_name)
    test_data = test_all_data['feat']
   
    #test_data_final = np.concatenate ((test_data,test_data_2),axis = 2)
    test_labels = test_all_data['label']
        
    totalframe = test_data.shape
    totalframe = totalframe[0]
    att_res['name'] = file_name
    #print(totalframe)
    pred_list= np.zeros((n_frames,n_classes))
    batch_xc = np.zeros((1,n_frames,test_data.shape[1]))
    batch_xc[0,:,:] = test_data
                 
    [pred,matte] = sess.run([soft_pred,matt], feed_dict={x: batch_xc, y:test_labels})
   
    pred_list    = pred
    matt_list    = matte
    

    label_index  = np.where(test_labels==1)[1][0]
    
    avg_pre = np.mean(pred_list, axis=0).tolist()
    top1 = (avg_pre.index(max(avg_pre))==label_index)
    top1_list.append(top1)
    att_res['pred_list'] = avg_pre
    att_res['label'] = test_labels 
    att_res['matt'] = matt_list
  
    print('video number:' ,num_batch, ' of ', test_num)
    print('current mAP = ',np.mean(top1_list))
    
    sio.savemat(output_path+file_name+'.mat',att_res, do_compression=False)

/usr/local/lib/python2.7/dist-packages/tensorflow/python/client/session.py:1645: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


./multi_model/multi_att_flow_1_ff/final_model
INFO:tensorflow:Restoring parameters from ./multi_model/multi_att_flow_1_ff/final_model
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_ba_med_3
('video number:', 0, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_1
('video number:', 1, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_goo_2
('video number:', 2, ' of ', 1529)
('current mAP = ', 1.0)
Aussie_Brunette_Brushing_Long_Hair_brush_hair_u_nm_np1_fr_med_0
('video number:', 3, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_fr_goo_0
('video number:', 4, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_le_goo_1
('video number:', 5, ' of ', 1529)
('current mAP = ', 1.0)
Brushing_Her_Hair__[_NEW_AUDIO_]_UPDATED_brush_hair_h_cm_np1_le_goo_2
('video number:', 6, ' of ', 1529)
('current m

('video number:', 68, ' of ', 1529)
('current mAP = ', 0.6521739130434783)
Goalkeeper_Training_Day__2_catch_f_cm_np1_ba_bad_5
('video number:', 69, ' of ', 1529)
('current mAP = ', 0.6428571428571429)
Goalkeeper_Training_Day__2_catch_f_cm_np1_fr_bad_0
('video number:', 70, ' of ', 1529)
('current mAP = ', 0.6338028169014085)
Goalkeeper_Training_Day__2_catch_f_cm_np1_fr_bad_1
('video number:', 71, ' of ', 1529)
('current mAP = ', 0.6388888888888888)
Goalkeeper_Training_Day__2_catch_f_cm_np1_fr_bad_2
('video number:', 72, ' of ', 1529)
('current mAP = ', 0.6301369863013698)
Goalkeeper_Training_Day__7_catch_f_cm_np1_ba_bad_2
('video number:', 73, ' of ', 1529)
('current mAP = ', 0.6216216216216216)
Goalkeeper_Training_Day__7_catch_f_cm_np1_le_bad_3
('video number:', 74, ' of ', 1529)
('current mAP = ', 0.6133333333333333)
Goalkeeper_Training_Day__7_catch_f_cm_np1_ri_med_0
('video number:', 75, ' of ', 1529)
('current mAP = ', 0.618421052631579)
Goalkeeper_Training_Day__7_catch_f_cm_np1_ri

('video number:', 138, ' of ', 1529)
('current mAP = ', 0.5107913669064749)
The_Slow_Clap_clap_u_nm_np1_fr_med_1
('video number:', 139, ' of ', 1529)
('current mAP = ', 0.5071428571428571)
The_Slow_Clap_clap_u_nm_np1_fr_med_12
('video number:', 140, ' of ', 1529)
('current mAP = ', 0.5035460992907801)
The_Slow_Clap_clap_u_nm_np1_fr_med_14
('video number:', 141, ' of ', 1529)
('current mAP = ', 0.5)
The_Slow_Clap_clap_u_nm_np1_fr_med_2
('video number:', 142, ' of ', 1529)
('current mAP = ', 0.4965034965034965)
The_Slow_Clap_clap_u_nm_np1_fr_med_24
('video number:', 143, ' of ', 1529)
('current mAP = ', 0.5)
The_Slow_Clap_clap_u_nm_np2_fr_bad_26
('video number:', 144, ' of ', 1529)
('current mAP = ', 0.503448275862069)
The_Slow_Clap_clap_u_nm_np2_fr_med_22
('video number:', 145, ' of ', 1529)
('current mAP = ', 0.5068493150684932)
The_Slow_Clap_clap_u_nm_np2_fr_med_8
('video number:', 146, ' of ', 1529)
('current mAP = ', 0.5034013605442177)
The_Slow_Clap_clap_u_nm_np3_fr_med_11
('video 

('video number:', 197, ' of ', 1529)
('current mAP = ', 0.47474747474747475)
Pole_Climbing_Challenging_gravity_climb_f_cm_np1_le_med_1
('video number:', 198, ' of ', 1529)
('current mAP = ', 0.47738693467336685)
Rock_Climbing_-_Joshua_Tree_climb_f_nm_np1_ba_goo_0
('video number:', 199, ' of ', 1529)
('current mAP = ', 0.475)
Rock_Climbing_-_Joshua_Tree_climb_f_nm_np1_ba_med_1
('video number:', 200, ' of ', 1529)
('current mAP = ', 0.472636815920398)
Rock_Climbing_-_Joshua_Tree_climb_f_nm_np1_ba_med_2
('video number:', 201, ' of ', 1529)
('current mAP = ', 0.47029702970297027)
Seamus_OD_arms_exploding-_rope_climb_climb_f_cm_np1_ba_med_0
('video number:', 202, ' of ', 1529)
('current mAP = ', 0.4729064039408867)
Speed_Climbing_Contest_Sport_Schuster_M_nchen_climb_f_cm_np1_ba_med_0
('video number:', 203, ' of ', 1529)
('current mAP = ', 0.47058823529411764)
Speed_Climbing_Contest_Sport_Schuster_M_nchen_climb_f_cm_np1_ba_med_1
('video number:', 204, ' of ', 1529)
('current mAP = ', 0.47317

...

('video number:', 1193, ' of ', 1529)
('current mAP = ', 0.4899497487437186)
SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_med_4
('video number:', 1194, ' of ', 1529)
('current mAP = ', 0.4895397489539749)
SMILES_FROM_OFF_THE_ROAD_3_-TRAVELS_WITH_A_HAT_smile_h_nm_np1_fr_med_5
('video number:', 1195, ' of ', 1529)
('current mAP = ', 0.4891304347826087)
Two_Towers_3_smile_h_nm_np1_fr_goo_7
('video number:', 1196, ' of ', 1529)
('current mAP = ', 0.48872180451127817)
showyoursmile_smile_h_nm_np1_fr_goo_0
('video number:', 1197, ' of ', 1529)
('current mAP = ', 0.48831385642737896)
smile_collection_7_smile_h_nm_np1_fr_goo_0
('video number:', 1198, ' of ', 1529)
('current mAP = ', 0.48790658882402)
smile_collection_7_smile_h_nm_np1_fr_goo_1
('video number:', 1199, ' of ', 1529)
('current mAP = ', 0.4875)
A_Beautiful_Mind_1_smoke_h_nm_np1_fr_goo_8
('video number:', 1200, ' of ', 1529)
('current mAP = ', 0.4870940882597835)
A_Beautiful_Mind_2_smoke_u_cm_np1_fr_goo_0
('video

('video number:', 1261, ' of ', 1529)
('current mAP = ', 0.48811410459587956)
AmericanGangster_stand_f_nm_np1_le_med_33
('video number:', 1262, ' of ', 1529)
('current mAP = ', 0.48772763262074426)
AmericanGangster_stand_u_cm_np1_fr_bad_0
('video number:', 1263, ' of ', 1529)
('current mAP = ', 0.4873417721518987)
AmericanGangster_stand_u_cm_np1_fr_bad_19
('video number:', 1264, ' of ', 1529)
('current mAP = ', 0.48695652173913045)
AmericanGangster_stand_u_nm_np1_fr_med_55
('video number:', 1265, ' of ', 1529)
('current mAP = ', 0.48657187993680884)
BIG_FISH_stand_u_nm_np1_fr_med_27
('video number:', 1266, ' of ', 1529)
('current mAP = ', 0.48697711128650356)
BLACK_HAWK_DOWN_stand_f_nm_np1_ba_med_14
('video number:', 1267, ' of ', 1529)
('current mAP = ', 0.4865930599369085)
BLACK_HAWK_DOWN_stand_u_cm_np1_fr_med_35
('video number:', 1268, ' of ', 1529)
('current mAP = ', 0.48699763593380613)
BLACK_HAWK_DOWN_stand_u_nm_np1_le_med_11
('video number:', 1269, ' of ', 1529)
('current mAP = 

('video number:', 1332, ' of ', 1529)
('current mAP = ', 0.477119279819955)
Kendo_Basics_sword_exercise_u_cm_np1_fr_bad_1
('video number:', 1333, ' of ', 1529)
('current mAP = ', 0.4775112443778111)
MAF_Tenshin_Ryu_sword_exercise_f_nm_np1_ri_bad_2
('video number:', 1334, ' of ', 1529)
('current mAP = ', 0.4771535580524345)
MAF_Tenshin_Ryu_sword_exercise_f_nm_np2_fr_bad_4
('video number:', 1335, ' of ', 1529)
('current mAP = ', 0.47679640718562877)
Spanish_counters_to_Italian_fencing_sword_exercise_f_cm_np2_le_bad_2
('video number:', 1336, ' of ', 1529)
('current mAP = ', 0.47718773373223633)
Spanish_counters_to_Italian_fencing_sword_exercise_f_cm_np2_ri_bad_1
('video number:', 1337, ' of ', 1529)
('current mAP = ', 0.476831091180867)
VERGIL_DEVIL_MAY_CRY_swinging_his_sword_sword_exercise_f_cm_np1_fr_med_1
('video number:', 1338, ' of ', 1529)
('current mAP = ', 0.4764749813293503)
VERGIL_DEVIL_MAY_CRY_swinging_his_sword_sword_exercise_f_cm_np1_fr_med_3
('video number:', 1339, ' of ', 1

('video number:', 1397, ' of ', 1529)
('current mAP = ', 0.4685264663805436)
Prelinger_FamilyLi1949_talk_u_nm_np1_fr_med_20
('video number:', 1398, ' of ', 1529)
('current mAP = ', 0.46890636168691924)
Prelinger_FamilyLi1949_talk_u_nm_np1_fr_med_3
('video number:', 1399, ' of ', 1529)
('current mAP = ', 0.4685714285714286)
Prelinger_FamilyLi1949_talk_u_nm_np1_ri_med_23
('video number:', 1400, ' of ', 1529)
('current mAP = ', 0.4689507494646681)
TheBoondockSaints_talk_h_cm_np1_fr_med_12
('video number:', 1401, ' of ', 1529)
('current mAP = ', 0.4686162624821683)
The_Fugitive_5_talk_h_nm_np1_fr_goo_10
('video number:', 1402, ' of ', 1529)
('current mAP = ', 0.4682822523164647)
The_Fugitive_5_talk_u_nm_np1_fr_goo_11
('video number:', 1403, ' of ', 1529)
('current mAP = ', 0.46794871794871795)
The_Matrix_2_talk_h_nm_np1_fr_goo_2
('video number:', 1404, ' of ', 1529)
('current mAP = ', 0.46832740213523133)
Two_Towers_5_talk_h_nm_np1_fr_goo_2
('video number:', 1405, ' of ', 1529)
('current m

('video number:', 1467, ' of ', 1529)
('current mAP = ', 0.45776566757493187)
likebeckam_turn_f_cm_np1_ri_med_43
('video number:', 1468, ' of ', 1529)
('current mAP = ', 0.45745405037440434)
likebeckam_turn_f_nm_np1_fr_med_38
('video number:', 1469, ' of ', 1529)
('current mAP = ', 0.45714285714285713)
AmericanGangster_walk_f_cm_np1_le_bad_8
('video number:', 1470, ' of ', 1529)
('current mAP = ', 0.4568320870156356)
AmericanGangster_walk_f_nm_np1_ba_med_72
('video number:', 1471, ' of ', 1529)
('current mAP = ', 0.45720108695652173)
AmericanGangster_walk_f_nm_np1_fr_med_38
('video number:', 1472, ' of ', 1529)
('current mAP = ', 0.4575695858791582)
AmericanGangster_walk_f_nm_np1_fr_med_6
('video number:', 1473, ' of ', 1529)
('current mAP = ', 0.45793758480325647)
AmericanGangster_walk_u_cm_np1_ba_med_37
('video number:', 1474, ' of ', 1529)
('current mAP = ', 0.4583050847457627)
AmericanGangster_walk_u_nm_np1_fr_med_15
('video number:', 1475, ' of ', 1529)
('current mAP = ', 0.458672

In [11]:
test_labels

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.]])

In [12]:
np.where(test_labels==1)[1][0]

50

In [13]:
sum(top1_list)

688